In [47]:
import pandas as pd
import os
import sys
from SStack import SStack
from queue import Queue,LifoQueue
import json
import copy
from py2neo import Graph,Node,Relationship,cypher

test_graph = Graph('http://localhost:7474',username='neo4j',password='8611662')

# 获取指定目录下的json文件
def get_files(path='D:\zyt\\azyt\sfx', rule=".json"):
    all = []
    for fpathe,dirs,fs in os.walk(path):   # os.walk获取所有的目录
        for f in fs:
            filename = os.path.join(fpathe,f)
            if filename.endswith(rule):  # 判断是否是".json"结尾
                all.append(filename)
    return all

'''
维护一个键的队列，输入参数为当前的属性名key和当前的队列的状态key_queue
当遇到key为下面的值的时候，不仅弹出当前的key，还弹出其前一个即父类的key
'''
def preserve_key_queue(key, key_queue):
    if key=='任免_辞职':
        delete = key_queue.get()
        delete = key_queue.get()
    elif key=='社会兼职' or key=='人才培养类' or key=='博士后':
        delete = key_queue.get()
    else:
        pass

'''
本函数用于处理倒数第二层属性并将其结果保存成dataframe
接受参数为维护的栈和队列，以及对应的院士名
当属性队列为空的时候，返回0
'''
def json_to_dataframe(st, key_queue, yuanshi_name_id):
    if not st.is_empty():
        content = set()
        word = st.pop() #弹出当前栈里面顶上的内容
        key_queue.put(word) #键值队列，用于保存前面已出现的键值(目的是方便后面迭代时能准确找到进行到什么地方了)
        attribute_value_list = [] #局部列表变量，用于存储不同百科文件当前属性下的值
        key_str = 'json_file'
        for j in key_queue.queue: #通过维护的键的队列取到对应的键值
            key_str += '[\''+j+'\']'
        #print("  ",key_str)
        attribute_value_list.append(eval(key_str))
        flag = 0 #标记这几个百科json文件的当前属性值是否为空
        for i in attribute_value_list:
            if type(i).__name__ == 'list': #处理百科json文件中该属性下的值如果是列表时的情况，则取出列表里的每一个字典
                flag = 1
                if word=='人物影响' or word=='研究领域': #这两个属性的值没有子属性了，其值直接就是字符串形式，所以单独处理
                    res = process_string(word, i, yuanshi_name_id)
                    key = key_queue.get()
                    preserve_key_queue(key, key_queue)
                    json_to_dataframe(st, key_queue, yuanshi_name_id)
                elif word=='人才培养类':
                    res = process_list_no_same(word, i, yuanshi_name_id) #返回的是一个列表，列表里面是多个字典
                    key = key_queue.get()
                    preserve_key_queue(key, key_queue) #得到动态维护的键的队列
                    json_to_dataframe(st, key_queue, yuanshi_name_id)
                else: #其他情况的属性值里还有子属性，此时如果是列表的话也是单独处理
                    res = process_list(word, i, yuanshi_name_id) #返回的是一个列表，列表里面是多个字典
                    key = key_queue.get()
                    preserve_key_queue(key, key_queue) #得到动态维护的键的队列
                    json_to_dataframe(st, key_queue, yuanshi_name_id)
                break
            elif type(i).__name__ == 'dict': #处理百科json文件中该属性下的值如果是字典时的情况，这里主要是针对单属性值
                flag = 1
                except_word = ['人物履历','教育经历','工作经历','其他职务','主要成果']
                if word not in except_word:
                    res = process_dict(word, i, yuanshi_name_id)
                    key = key_queue.get()
                    preserve_key_queue(key, key_queue)
                    json_to_dataframe(st, key_queue, yuanshi_name_id)
                else: #当不是单属性值时，说明其还不是最外层的属性(也即为父属性)，则将其压入栈中后续出栈时再处理
                    dict_key1 = list(i.keys()) #将字典里的键都放入栈中，稍后依次取出进行对齐
                    dict_key1.reverse()
                    for i in dict_key1:
                        st.push(i)
                    json_to_dataframe(st, key_queue, yuanshi_name_id)
                break
            else:
                pass
        if flag==0: #表示这几个文件的当前属性都为空
            key = key_queue.get()
            preserve_key_queue(key, key_queue)
            json_to_dataframe(st, key_queue, yuanshi_name_id)
    else:
        return 0

''' 
用于查看当前的值是否已经存在于原dataframe里了，若存在则查看是否已经被连线过了
若没有连线，则添加一条边，若有连线则依次拥有相同值的下一个，若都有连线，则新加入一行
'''
def line2frame(word, content, yuanshi_name_id):
    global used_node_flag
    global count_id
    global used_node_flag #表示已使用过的节点集合
    global node
    global relation
    if content in node['name'].values: #如果当前处理的值在已有的dataframe已经存在
        relative_content_line = node.loc[node['name']==content] #在原有dataframe里找出所有该值对应的行
        flag = 0
        for i in relative_content_line['id'].values:
            if i in used_node_flag: #如果该id已经被处理过，则查看下一个
                continue
            else:
                new_relation_dict = {'from_id':yuanshi_name_id, 'pro1':word, 'to_id':i}
                new_relation_line = pd.DataFrame(new_relation_dict,index=[0])
                relation = relation.append(new_relation_line,ignore_index=True)
                used_node_flag.append(i) #标志为已被处理过
                flag = 1 #表示找到了没被处理过的有相同值的行
                break
        if flag==0: #表示所有该值对应的行都被处理过
            write_to_dataframe(word, content, yuanshi_name_id, count_id)
            count_id += 1
        else:
            pass
    else:
        write_to_dataframe(word, content, yuanshi_name_id, count_id)
        count_id += 1
    
# 用于将处理结果存入dataframe里
def write_to_dataframe(word, content, yuanshi_name_id, to_id):
    global node
    global relation
    new_node_dict = {'id':to_id, 'name':content}
    new_node_line = pd.DataFrame(new_node_dict,index=[0])
    new_relation_dict = {'from_id':yuanshi_name_id, 'pro1':word, 'to_id':to_id}
    new_relation_line = pd.DataFrame(new_relation_dict,index=[0])
    node = node.append(new_node_line,ignore_index=True)
    relation = relation.append(new_relation_line,ignore_index=True)
    
#处理人物影响、研究领域时转化为dataframe的情况 
def process_string(word, attribute_value, yuanshi_name_id):
    content = ''
    for i in attribute_value:
        content += i
    line2frame(word, content, yuanshi_name_id)

# 处理人才培养里面有教育理念和指导学生这两种情况
def process_list_no_same(word, attribute_value_list, yuanshi_name_id):
    if len(attribute_value_list)==2:
        content1 = attribute_value_list[0]['教育理念']
        content2 = attribute_value_list[1]['指导学生']
        line2frame('教育理念', content1, yuanshi_name_id)
        line2frame('指导学生', content2, yuanshi_name_id)
    else:
        if '教育理念' in attribute_value_list[0].keys():
            content = attribute_value_list[0]['教育理念']
            line2frame('教育理念', content, yuanshi_name_id)
        if '指导学生' in attribute_value_list[0].keys():
            content = attribute_value_list[0]['指导学生']
            line2frame('指导学生', content, yuanshi_name_id)
    
# 处理其他除人物影响、研究领域外的list的情况
def process_list(word, attribute_value_list, yuanshi_name_id):
    keys = attribute_value_list[0].keys()
    list_key = ['所获奖项','授予奖项名称','论著名称','散文名称','项目名称','研究成果名称','专利名称','教育理念','指导学生','事件','语录','争议起因','事件起因','方向']
    for i in attribute_value_list:
        content = ''
        if '组织机构' in keys:
            if i['组织机构']=='' and i['职位_职称']=='':
                continue
            content = i['组织机构'] + i['职位_职称']
            line2frame(word, content, yuanshi_name_id)
        elif '任免职位_职称' in keys:
            if i['任免职位_职称']=='':
                continue
            content = i['信息公布权威机关'] + i['任免职位_职称']
            line2frame(word, content, yuanshi_name_id)
        elif '所在单位' in keys:
            if i['所在单位']=='' and i['职位_职称']=='':
                continue
            content = i['所在单位'] + i['职位_职称']
            line2frame(word, content, yuanshi_name_id)
        elif '演讲_报告题目' in keys:
            if i['组织单位_活动单位名称']=='' and i['演讲_报告题目']=='':
                continue
            content = i['组织单位_活动单位名称'] + i['演讲_报告题目']
            line2frame(word, content, yuanshi_name_id)
        else:
            for key in keys:
                if key in list_key:
                    if i[key]=='':
                        continue
                    content = i[key]
                    line2frame(key, content, yuanshi_name_id)

# 用于处理字典的情况
def process_dict(word, attribute_value_list, yuanshi_name_id):
    keys = attribute_value_list.keys()
    key_list = ['外文名','性别','出生日期','出生地','国籍','民族','职业','毕业院校','政治面貌','代表作品','主要成就','曾任职','信仰','原籍','学校名称']
    for key in keys:
        content = ''
        if (key in key_list) and (type(attribute_value_list[key]).__name__ == 'dict'):
            content = attribute_value_list[key][key]
            if word=='基本信息':
                line2frame(key, content, yuanshi_name_id)
            else:
                line2frame(word, content, yuanshi_name_id)
        elif (key in key_list) and (type(attribute_value_list[key]).__name__ == 'list'):
            for i in attribute_value_list[key]:
                if word=='基本信息':
                    content = i[key]
                    line2frame(key, content, yuanshi_name_id)
                else:
                    content = i[key]
                    line2frame(word, content, yuanshi_name_id)
        else:
            pass

def process_entrace(key_list, yuanshi_name_id):
    st = SStack() #用栈存放每一轮的属性
    lq = LifoQueue(maxsize=0) #用后进先出队列用于存放文件操作过程中的中间键值(用于新建新的对齐json)
    key_list.reverse() #属性名列表反向，为了使最后按照正向的方式写入
    for i in key_list:
        st.push(i) #将最开始外层的属性名压入栈中
    res = json_to_dataframe(st, lq, yuanshi_name_id) #调用处理成dataframe的函数
    global node
    global relation
    node.to_csv('E://neo4j/import/fang_node2.csv',index=False) #将df输出到csv文件，输出顺序为dataframe默认的列名顺序
    relation.to_csv('E://neo4j/import/fang_relation2.csv',index=False)
    # 表示先删除当前已有的节点和关系
    cypher_delete = '''MATCH(b) detach delete b
                  '''
    test_graph.run(cypher_delete)
    # 表示加载节点到neo4j
    cypher_node = '''LOAD CSV WITH HEADERS FROM "file:///fang_node2.csv" AS line
                     MERGE (p:test{id:line.id,name:line.name})
                  '''
    test_graph.run(cypher_node)
    # 表示加载关系到neo4j
    cypher_relation = '''LOAD CSV WITH HEADERS FROM "file:///fang_relation2.csv" AS line  
                         match (from:test{id:line.from_id}),(to:test{id:line.to_id})  
                         merge (from)-[r:rel{pro1:line.pro1}]->(to)
                      '''
    test_graph.run(cypher_relation)

def deal_yuanshinameid_and_in_process_entrnce(every_baike_json, json_file):
    global count_id
    global used_node_flag
    global node
    global relation
    used_node_flag = [] #定义一个列表，记录已使用节点
    key_list = list(json_file.keys()) #局部变量，用于存放百科json文件的最外层父属性(院士名和百科名不计算)
    yuanshi_name = every_baike_json['院士名']
    last_line_value = node['id'][-1:].values[0] #取出当前已存在csv文件的最后一行节点的id号
    count_id = last_line_value + 1 #由该id号的下一个数后开始进行增加
    #院士名这个节点单独处理
    new_node = pd.DataFrame()
    new_node["id"] = [count_id]
    new_node["name"] = [yuanshi_name]
    node = pd.concat([node, new_node], axis=0, ignore_index=True)
    #院士中文名这个关系单独处理
    new_relation = pd.DataFrame()
    new_relation["from_id"] = [count_id]
    new_relation["pro1"] = ['中文名']
    new_relation["to_id"] = [count_id]
    relation = pd.concat([relation, new_relation], axis=0, ignore_index=True)
    yuanshi_name_id = count_id #将院士名这个ID保存下来
    count_id += 1
    process_entrace(key_list, yuanshi_name_id)

if __name__ == "__main__":
    global node
    global relation
    global json_file #全局列表变量，用于存放不同百科json文件的内容
    node = pd.read_csv("./node.csv")
    relation = pd.read_csv("./relation.csv")
    b = get_files(r"./")
    for i in b:
        if i.split('./')[1]!='fang_final.json':
            with open(i,'r',encoding='utf-8') as f: #打开json文件并保存到json文件列表里
                every_baike_json = json.load(f)
                json_file = every_baike_json
            deal_yuanshinameid_and_in_process_entrnce(every_baike_json, json_file)

In [10]:
import pandas as pd
node = pd.read_csv("./node.csv")
last_line_value = node['id'][-1:].values[0]

   <class 'numpy.int64'>


In [ ]:
# 删除：
# 删除所有节点和关系：
# START n=node(*) 
# match n-[r]-()
# delete n,r;
# 删除所有节点以上方法过时，后面版本将被遗弃-推荐使用如下方法
# MATCH (n)
# OPTIONAL MATCH (n)-[r]-()
# DELETE n,r

# MATCH(b) detach delete b

In [21]:
for i in node['name'][1:]:
    print("  ",i)
    pass
if '中国工程院院士' in node['name'].values:
    print("  True")

   北京邮电大学校长
   计算机系助教/讲师/副教授
   计算机系教授
   哈尔滨工业大学计算机与电气工程学院副院长
   中国国家计算机网络与信息安全管理中心副总工程师
   国家计算机网络应急技术处理协调中心总工程师/副主任/教授级高级工程师
   国家计算机网络应急技术处理协调中心主任/总工程师/教授级高级工程师
   信息产业部互联网应急处理协调办公室主任
   国家计算机网络与信息安全管理中心名誉主任
   北京邮电大学校长
   哈尔滨工业大学计算机系计算机系统结构教研室副主任、主任
   哈尔滨工业大学网络中心主任
   中国工程院院士
   国家计算机网络与信息安全管理中心名誉主任
   中国网络空间安全协会理事长
   哈尔滨工业大学（深圳）计算机科学与技术学院教授、首席学术顾问
   广州大学网络空间先进技术研究院名誉院长
   哈尔滨工业大学
   国家计算机网络与信息安全管理中心总工程师、副主任、教授级高级工程师
   中国科学院计算技术研究所客座研究员、博士生导师、信息安全首席科学家
   国家计算机网络与信息安全管理中心主任、总工程师、教授级高级工程师、名誉主任
   国防科学技术大学兼职教授、特聘教授、博士生导师
   第十一届全国人大第十一届全国人大代表
   北邮基层人大代表
   中国科学院信息工程研究所客座研究员、博士生导师
   中国科学院信息工程研究所学术委员会主任
   中国网络空间安全协会第一届理事会理事长
   哈尔滨工业大学计算机系助教/讲师/副教授/教研室副主任
   哈尔滨工业大学计算机系教授
   哈尔滨工业大学计算机系博士生导师
   国家计算机网络应急技术处理协调中心
   国防科学技术大学
   哈尔滨工业大学
   哈尔滨工业大学
   清华大学
   中国通信标准化协会理事
   中国互联网协会副理事长
   网络与信息安全工作委员会主任
   中国通信学会学术工作委员会委员
   清华大学计算机系教授
   中科院计算所客座研究员/博士生导师/信息安全首席科学家
   国防科学技术大学教授/博士生导师
   《通信学报》编辑委员会主任
   中国计算机学会副理事长/计算机安全专业委员会主任
   信息产业部通信科学技术委员会常务委员
   哈尔滨工业大学教授、博士生导师
   哈尔滨工业大学

In [33]:
temp = node.loc[node['name']=='哈尔滨工业大学']
for i  in temp['id'].values:
    print("  ",i)

   19
   34
   35
   132
